# Getting started with scikit-learn


## Loading packages

In [ ]:
!pip install scikit-learn==0.23.2

## !!! Restart kernel to use above version of scikit-learn

In [ ]:
# general
import math
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
plt.style.use('seaborn-whitegrid')
%matplotlib inline

# scipy
from scipy import stats

# scikit-learn
from sklearn import datasets
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## Fitting a model without scikit-learn

An example with [generated regression data](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html):

In [ ]:
# create some data and create a scatter plot
X, y, coef = datasets.make_regression(n_samples=100, n_features=1, noise=50, random_state=0, coef=True)

plt.plot(X, y, '.', color='grey');

Fit a line through the points:

$y = a + b*x$

where $a$ the intercept (the value of $y$ where $x=0$) and $b$ is the gradient of the line. 

This is actually a straightforward problem that can be solved directly with linear algebra. From this [example](https://machinelearningmastery.com/solve-linear-regression-using-linear-algebra/):

$X^T . X . b = X^T . y$

$b = (X^T . X)^-1 . X^T . y$

Note that it is assumed here that $a=0$

*[This course](https://www.coursera.org/learn/machine-learning) by Andrew Ng goes into much more detail.*

In [ ]:
b = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(b)

In [ ]:
yhat = X.dot(b)

In [ ]:
plt.plot(X, y, '.', color='grey');
plt.plot(X, yhat, color='red')

Another way to solve this is to use the mean difference between the line and all points using the [least squares method](https://en.wikipedia.org/wiki/Least_squares), where you want to minimize (find the smallest possible value) for the root means square error:

$RMSE = \sqrt{ \frac{1}{N}\sum_{i=1}^{N} (\hat{y}_{i} - y_{i})^2}$

where $y_{i}$ is the observed value and $\hat{y}_{i}$ the simulated value for each point $i$ and $N$ the number of data points.

In [ ]:
RMSE = np.sqrt(sum(np.power(yhat-y,2))/len(y))
RMSE

## Fitting a model with [SciPy](https://docs.scipy.org/doc/scipy/reference/)

This package contains various [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html), including the [least squares method](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#least-squares-minimization-least-squares) that can be used to fit the above line:

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(X[:,0], y)
print("slope: %f    intercept: %f" % (slope, intercept))
print("r_squared: %f    std_err: %f" % (r_value**2, std_err))

`r_squared` is the coefficient of determination

`std_err` is the [standard error](https://en.wikipedia.org/wiki/Standard_error)



In [ ]:
plt.plot(X, y, '.', color='grey');
plt.plot(X, yhat, color='red')
plt.plot(X, intercept + slope*X, 'blue')

## Fitting a model with Scikit-learn

When the data becomes more complex, e.g. many more variables, non-linear, including binary or multiple classes fitting a 'line' through the data is less straightforward. 

The principle stays the same: find the best 'line' through your data points. This 'line' is the machine learning model.  

With scikit-learn:

In [ ]:
linreg = linear_model.LinearRegression()

linreg.fit(X,y)

ypred= linreg.predict(X)

print(linreg)
print('Intercept: \n',linreg.intercept_)
print('Coefficients: \n', linreg.coef_)

In [ ]:
plt.plot(X, y, '.', color='grey');
plt.plot(X, yhat, color='red')
plt.plot(X, intercept + slope*X, 'blue')
plt.plot(X, ypred, color='green')

Above a line was fitted in 3 different ways, with a few metrics calculated. This is only one part of the process when building and testing a model. A complete workflow consists of these steps:

* Data exploration
* Data preprocessing
* Splitting data for training and testing
* Preparing a model
* Assembling all of the steps using pipeline
* Training the model
* Running predictions on the model
* Evaluating and visualizing model performance

Ideally you would go through each of these steps one by one, but in practice you will go back and forth between them when you start to understand the data better.

# Full example

## [Diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset)

An example with data provided with scikit-learn:

* Ten baseline variables: age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients, as well as the response of interest, a quantitative measure of disease progression one year after baseline
* Each of these 10 feature variables have been mean centered and scaled by the standard deviation times n_samples (i.e. the sum of squares of each column totals 1)
* The target is a quantitative measure of disease progression one year after baseline

In [ ]:
# as_frame=True loads the data as a pandas DataFrame (X) and Series (y)
diabetes = datasets.load_diabetes(as_frame=True)

X = diabetes.data
y = diabetes.target

feature_names = diabetes.feature_names
print(feature_names)

In [ ]:
X.to_csv('diabetes.csv')

In [ ]:
X.insert(0, "target", y)
sns.pairplot(X[['target','age', 'sex', 'bmi', 'bp']], kind='reg', diag_kind='kde');

In [ ]:
sns.pairplot(X[['target','s1', 's2', 's3', 's4', 's5', 's6']], kind='reg', diag_kind='kde');

In [ ]:
X.corr().style.background_gradient(cmap='coolwarm')

3 groups of data:

* `age`, `bmi` and `bp` - continuous
* `sex` - 2 classes (0 or 1)
* `s1`,`s2`,`s3`,`s4`,`s5` and `s6` - seem strongly correlated, continuous
* all are normalized, so this step can be skipped

In [ ]:
# Use one features
X1 = X[['age']]

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.25, random_state=42)

# Create linear regression object
regr1 = linear_model.LinearRegression()

# Train the model using the training data
regr1.fit(X1_train, y1_train)

# Make predictions using the test data
y1_pred = regr1.predict(X1_test)

print('Coefficients: \n', regr1.coef_)
print('RMSE: %.2f' % np.sqrt(mean_squared_error(y1_test, y1_pred)))
print('Coefficient of determination (r2): %.2f' % r2_score(y1_test, y1_pred))

plt.plot(X1_train, y1_train, '.', color='grey');
plt.plot(X1_test, y1_pred, '.', color='red');

In [ ]:
# Use three features
X2 = X[['age', 'bmi', 'bp']]

# Split the data into training/testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.25, random_state=42)

# Create linear regression object
regr2 = linear_model.LinearRegression()

# Train the model using the training data
regr2.fit(X2_train, y2_train)

# Make predictions using the test data
y2_pred = regr2.predict(X2_test)

print('Coefficients: \n', regr2.coef_)
print('RMSE: %.2f' % np.sqrt(mean_squared_error(y2_test, y2_pred)))
print('Coefficient of determination: %.2f' % r2_score(y2_test, y2_pred))

plt.plot(X[['age']], y, '.', color='grey');
plt.plot(X2_test[['age']], y2_pred, '.', color='red');

In [ ]:
plt.plot(y2_test, y2_pred, '.', color='red');

In [ ]:
# Use the blood measurements
X3 = X[['s1', 's2', 's3', 's4', 's5', 's6']]

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.25, random_state=42)

# Create linear regression object
regr3 = linear_model.LinearRegression()

# Train the model using the training data
regr3.fit(X3_train, y3_train)

# Make predictions using the test data
y3_pred = regr3.predict(X3_test)

print('Coefficients: \n', regr3.coef_)
print('Mean squared error: %.2f' % mean_squared_error(y3_test, y3_pred))
print('Coefficient of determination: %.2f' % r2_score(y3_test, y3_pred))

plt.plot(y3_test, y3_pred, '.', color='grey');

In [ ]:
# Use all data
X4 = X[['age', 'bmi', 'sex', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']]

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y, test_size=0.25, random_state=42)

# Create linear regression object
regr4 = linear_model.LinearRegression()

# Train the model using the training data
regr4.fit(X4_train, y4_train)

# Make predictions using the test data
y4_pred = regr4.predict(X4_test)

print('Coefficients: \n', regr4.coef_)
print('RMSE: %.2f' % np.sqrt(mean_squared_error(y4_test, y4_pred)))
print('Coefficient of determination (r2): %.2f' % r2_score(y4_test, y4_pred))

plt.plot(y4_test, y4_pred, '.', color='grey');

In [ ]:
# Plot outputs
plt.scatter(X4_test['bmi'], y4_test, color='grey')
plt.scatter(X4_test['bmi'], y4_pred, color='red')

In [ ]:
# what we have so far

models = ['LinearRegression', 'LinearRegression', 'LinearRegression', 'LinearRegression']
r2s = [r2_score(y1_test, y1_pred),r2_score(y2_test, y2_pred),r2_score(y3_test, y3_pred),r2_score(y4_test, y4_pred)]
RMSEs = [np.sqrt(mean_squared_error(y1_test, y1_pred)),np.sqrt(mean_squared_error(y2_test, y2_pred)),
        np.sqrt(mean_squared_error(y3_test, y3_pred)),np.sqrt(mean_squared_error(y4_test, y4_pred))]

age = [regr1.coef_[0], regr2.coef_[0], float('NaN'), regr4.coef_[0]]
bmi = [float('NaN'), regr2.coef_[1], float('NaN'), regr4.coef_[1]]
bp = [float('NaN'), regr2.coef_[2], float('NaN'), regr4.coef_[3]]
sex = [float('NaN'), float('NaN'), float('NaN'), regr4.coef_[2]]
s1 = [float('NaN'), float('NaN'), regr3.coef_[0], regr4.coef_[4]]
s2 = [float('NaN'), float('NaN'), regr3.coef_[1], regr4.coef_[5]]
s3 = [float('NaN'), float('NaN'), regr3.coef_[2], regr4.coef_[6]]
s4 = [float('NaN'), float('NaN'), regr3.coef_[3], regr4.coef_[7]]
s5 = [float('NaN'), float('NaN'), regr3.coef_[4], regr4.coef_[8]]
s6 = [float('NaN'), float('NaN'), regr3.coef_[5], regr4.coef_[9]]

summary = pd.DataFrame(list(zip(models, r2s, RMSEs, age, bmi, bp, sex, s1, s2, s3, s4 ,s5, s6)), 
               columns =['model', 'r2', 'RMSE','age','bmi','bp','sex', 's1', 's2', 's3', 's4', 's5', 's6']) 

summary.transpose()

#### [Ridge regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) or $L_2$ regularization

* Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of the coefficients. 
* The complexity parameter $\alpha$ controls the amount of shrinkage: the larger the value of $\alpha$, the greater the amount of shrinkage and thus the coefficients become more robust to collinearity.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train = X_train.drop('target', axis=1)
X_test = X_test.drop('target', axis=1)

regr5 = linear_model.Ridge(alpha=0.2)
#regr5 = linear_model.Ridge(alpha=0.2,solver='lsqr')
regr5.fit(X_train, y_train)
y5_pred = regr5.predict(X_test)

summary.loc[len(summary)] = ['Ridge (alpha=0.2)',
                     r2_score(y_test, y5_pred),
                     np.sqrt(mean_squared_error(y_test, y5_pred)),
                     regr5.coef_[0],regr5.coef_[1],regr5.coef_[2],regr5.coef_[3],regr5.coef_[4],
                     regr5.coef_[5],regr5.coef_[6],regr5.coef_[7],regr5.coef_[8],regr5.coef_[9]]

summary.transpose()

#### [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso) or $L_1$ regularization

* The Lasso is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer non-zero coefficients, effectively reducing the number of features upon which the given solution is dependent. For this reason Lasso and its variants are fundamental to the field of compressed sensing

> Have a look at [this example](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py) for a method to 

In [ ]:
regr6 = linear_model.Lasso(alpha=0.1)
regr6.fit(X_train, y_train)
y6_pred = regr6.predict(X_test)

summary.loc[len(summary)] = ['Lasso (alpha=0.1)',
                     r2_score(y_test, y6_pred),
                     np.sqrt(mean_squared_error(y_test, y6_pred)),
                     regr6.coef_[0],regr6.coef_[1],regr6.coef_[2],regr6.coef_[3],regr6.coef_[4],
                     regr6.coef_[5],regr6.coef_[6],regr6.coef_[7],regr6.coef_[8],regr6.coef_[9]]

summary.transpose()

#### Feature selection

The Lasso regression can be used to perform [feature selection](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel). 

[Another example](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py).

In [ ]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(regr6, prefit=True)
print(X_train.shape)
X_train_new = model.transform(X_train)
X_test_new = model.transform(X_test)
X_train_new.shape

In [ ]:
model.get_support()

In [ ]:
print(feature_names)
new_feature_names = ['sex', 'bmi', 'bp', 's1', 's3', 's5', 's6']
new_feature_names

In [ ]:
# retrain model
regr7 = linear_model.Lasso(alpha=0.1)
regr7.fit(X_train_new, y_train)
y7_pred = regr7.predict(X_test_new)

summary.loc[len(summary)] = ['Lasso (alpha=0.1, 7 feat.)',
                     r2_score(y_test, y7_pred),
                     np.sqrt(mean_squared_error(y_test, y7_pred)),
                     float('NaN'),regr7.coef_[0],regr7.coef_[1],regr7.coef_[2],regr7.coef_[3],
                     float('NaN'),regr7.coef_[4],float('NaN'),regr7.coef_[5],regr7.coef_[6]]

summary.transpose()

In [ ]:
regr8 = linear_model.Ridge(alpha=0.2)
regr8.fit(X_train_new, y_train)
y8_pred = regr8.predict(X_test_new)

summary.loc[len(summary)] = ['Ridge (alpha=0.2, 7 feat.)',
                     r2_score(y_test, y8_pred),
                     np.sqrt(mean_squared_error(y_test, y8_pred)),
                     float('NaN'),regr8.coef_[0],regr8.coef_[1],regr8.coef_[2],regr8.coef_[3],
                     float('NaN'),regr8.coef_[4],float('NaN'),regr8.coef_[5],regr8.coef_[6]]

summary.transpose()

#### Hyperparameter optimization

In [ ]:
np.logspace(-3, 3, 40)

In [ ]:
from sklearn.model_selection import cross_val_score

alphas = np.logspace(-3, -1, 30)

for Model in [linear_model.Ridge, linear_model.Lasso]:
    scores = [cross_val_score(Model(alpha), X_train, y_train, cv=3).mean()
              for alpha in alphas]
    plt.plot(alphas, scores, label=Model.__name__) 

In [ ]:
from sklearn.model_selection import GridSearchCV
for Model in [linear_model.Ridge, linear_model.Lasso]:
    gscv = GridSearchCV(Model(), dict(alpha=alphas), cv=3).fit(X_train, y_train)
    print('%s: %s' % (Model.__name__, gscv.best_params_))

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV
for Model in [RidgeCV, LassoCV]:
    model = Model(alphas=alphas, cv=3).fit(X_train, y_train)
    print('%s: %s' % (Model.__name__, model.alpha_))

In [ ]:
for Model in [RidgeCV, LassoCV]:
    scores = cross_val_score(Model(alphas=alphas, cv=3), X_train, y_train, cv=3)
    print(Model.__name__, np.mean(scores))

#### [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html?highlight=decisiontreeregressor#sklearn.tree.DecisionTreeRegressor)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=3,random_state=42)
tree.fit(X_train, y_train)
tree_pred = tree.predict(X_test)

print(tree.feature_importances_)

summary.loc[len(summary)] = ['DecisionTree',
                     r2_score(y_test, tree_pred),
                     np.sqrt(mean_squared_error(y_test, tree_pred)),
                     float('NaN'),float('NaN'),float('NaN'),float('NaN'),float('NaN'),
                     float('NaN'),float('NaN'),float('NaN'),float('NaN'),float('NaN')]

summary.round(3)


### [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor)

#### [Polynomial features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html#)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
#poly = PolynomialFeatures(interaction_only=True)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)
print(X_train_poly.shape)

regr6 = linear_model.Lasso(alpha=0.1)
regr6.fit(X_train_poly, y_train)
y6_pred = regr6.predict(X_test_poly)


print(r2_score(y_test, y6_pred))
print(np.sqrt(mean_squared_error(y_test, y6_pred)))

regr6.coef_

## SHAP

[Example](https://shap.readthedocs.io/en/latest/example_notebooks/kernel_explainer/Diabetes%20regression.html)

In [ ]:
import shap
shap.initjs()

X_train_summary = shap.kmeans(X_train, 10)

def print_accuracy(f):
    print("Root mean squared test error = {0}".format(np.sqrt(np.mean((f(X_test) - y_test)**2))))
    time.sleep(0.5) # to let the print get out before any progress bars

lin_regr = linear_model.LinearRegression()
lin_regr.fit(X_train, y_train)

ex = shap.KernelExplainer(lin_regr.predict, X_train_summary)
shap_values = ex.shap_values(X_test.iloc[0,:])
shap.force_plot(ex.expected_value, shap_values, X_test.iloc[0,:])

In [ ]:
shap_values = ex.shap_values(X_test.iloc[2,:])
shap.force_plot(ex.expected_value, shap_values, X_test.iloc[2,:])

In [ ]:
shap_values = ex.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.dependence_plot("s1", shap_values, X_test)

# Learn more

* [Learn regression algorithms using Python and scikit-learn](https://developer.ibm.com/tutorials/learn-regression-algorithms-using-python-and-scikit-learn/)


